In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Agrega una dimension de canales
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Crea una instancia del modelo
model = MyModel()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [12]:
EPOCHS = 10

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Perdida: {}, Exactitud: {}, Perdida de prueba: {}, Exactitud de prueba: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reinicia métricas para el siguiente epoch
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()


Epoch 1, Perdida: 0.13678205013275146, Exactitud: 95.87833404541016, Perdida de prueba: 0.06106884777545929, Exactitud de prueba: 98.04000091552734
Epoch 2, Perdida: 0.043246954679489136, Exactitud: 98.68000030517578, Perdida de prueba: 0.047107063233852386, Exactitud de prueba: 98.40999603271484
Epoch 3, Perdida: 0.02141445316374302, Exactitud: 99.32666778564453, Perdida de prueba: 0.05560636147856712, Exactitud de prueba: 98.27999877929688
Epoch 4, Perdida: 0.014012225903570652, Exactitud: 99.54666137695312, Perdida de prueba: 0.06823136657476425, Exactitud de prueba: 98.04000091552734
Epoch 5, Perdida: 0.009848949499428272, Exactitud: 99.65666198730469, Perdida de prueba: 0.06430879235267639, Exactitud de prueba: 98.37999725341797
Epoch 6, Perdida: 0.006194631569087505, Exactitud: 99.80166625976562, Perdida de prueba: 0.06539874523878098, Exactitud de prueba: 98.47999572753906
Epoch 7, Perdida: 0.005712233018130064, Exactitud: 99.79666137695312, Perdida de prueba: 0.0729722157120704